In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import *

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# Carnê

In [ ]:
carne = sc.textFile('CARNE.txt')

In [ ]:
header = carne.first()
header

In [ ]:
#filter out the header, make sure the rest looks correct
carne = carne.filter(lambda line: line != header)

In [ ]:
#here's where the changes take place
#this creates a dataframe using whatever pyspark feels like using (I think string is the default). the header.split is providing the names of the columns
carne_df = carne.map(lambda k: k.split("\t")).toDF(header.split("\t"))
carne_df.show()

In [ ]:
carne_df_ts = carne_df.withColumn("DATA_VENCIMENTO", to_timestamp("DATA_VENCIMENTO", "yyyy/MM/dd HH:mm:ss"))
carne_df_ts = carne_df_ts.withColumn("DATA_PAGAMENTO", to_timestamp("DATA_PAGAMENTO", "yyyy/MM/dd HH:mm:ss"))
carne_df_ts = carne_df_ts.withColumn("DATA_CONTRATO", to_timestamp("DATA_CONTRATO", "yyyy/MM/dd HH:mm:ss"))

In [ ]:
carne_df_ts.show(5)

Total de clientes no carnê:

In [ ]:
carne_df_ts.select(carne_df_ts['CODIGO_CLIENTE']).dropDuplicates().count()

Dados faltantes:

In [ ]:
carne_df_ts.where(col("DATA_VENCIMENTO").isNull() | col("DATA_PAGAMENTO").isNull()).select(carne_df_ts['CODIGO_CLIENTE']).dropDuplicates().count()

# PAGOU ANTES DO VENCIMENTO

In [ ]:
possible_good = carne_df_ts.select(carne_df_ts['CODIGO_CLIENTE'],carne_df_ts['DATA_PAGAMENTO'],carne_df_ts['DATA_VENCIMENTO']).where(carne_df_ts['DATA_PAGAMENTO'] <= carne_df_ts['DATA_VENCIMENTO'])

In [ ]:
possible_client_carne = possible_good.select("CODIGO_CLIENTE").dropDuplicates()

Contagem de clientes "bons" no carnê:

In [ ]:
good_client_carne = possible_client_carne.subtract(bad_client_carne)

# PAGOU DEPOIS DO VENCIMENTO

In [ ]:
bad_carne = carne_df_ts.select(carne_df_ts['CODIGO_CLIENTE'],carne_df_ts['DATA_PAGAMENTO'],carne_df_ts['DATA_VENCIMENTO']).where(carne_df_ts['DATA_PAGAMENTO'] > carne_df_ts['DATA_VENCIMENTO'])

In [ ]:
bad_carne.show(5)

Contagem de pagamentos no carnê:

In [ ]:
bad_carne.count()

In [ ]:
bad_client_carne = bad_carne.select("CODIGO_CLIENTE").dropDuplicates()

Contagem de clientes "maus" no carnê:

In [ ]:
bad_client_carne.count()

In [ ]:
bad_client_carne = bad_client_carne.withColumn("CLASS", lit(0))
bad_client_carne.show()

Total_Carne = Good + Bad = 88829 + 516271 = 605100

# Cadastro

In [3]:
cadastral = sc.textFile('CADASTRAL.txt')

In [4]:
header1 = cadastral.first()
header1

'CODIGO_CLIENTE\tDATA_NASCIMENTO\tSEXO\tESTADO_CIVIL\tCIDADE\tUF\tBAIRRO\tCEP\tQTD_DEPENDENTES\tDATA_CADASTRO\tDATA_ADMISSAO\tCATEGORIAL_PROFISSAO\tTIPO_RESIDENCIA\tRENDA_TITULAR\tRENDA_CJ\tOUTRAS_RENDAS\tQTD_CARTOES_ADICIONAIS\tFLAG_CONTA_BANCO\tVALOR_HISTORICO_COMPRAS'

In [5]:
cadastral = cadastral.filter(lambda line: line != header1)

In [6]:
cadastral_df = cadastral.map(lambda k: k.split("\t")).toDF(header1.split("\t"))

In [7]:
cadastral_df = cadastral_df.withColumn("DATA_CADASTRO", to_timestamp("DATA_CADASTRO", "yyyy/MM/dd HH:mm:ss"))
cadastral_df.show(5)

+--------------+--------------------+----+------------+-------------+---+--------+--------+---------------+-------------------+--------------------+--------------------+---------------+-------------+--------+-------------+----------------------+----------------+-----------------------+
|CODIGO_CLIENTE|     DATA_NASCIMENTO|SEXO|ESTADO_CIVIL|       CIDADE| UF|  BAIRRO|     CEP|QTD_DEPENDENTES|      DATA_CADASTRO|       DATA_ADMISSAO|CATEGORIAL_PROFISSAO|TIPO_RESIDENCIA|RENDA_TITULAR|RENDA_CJ|OUTRAS_RENDAS|QTD_CARTOES_ADICIONAIS|FLAG_CONTA_BANCO|VALOR_HISTORICO_COMPRAS|
+--------------+--------------------+----+------------+-------------+---+--------+--------+---------------+-------------------+--------------------+--------------------+---------------+-------------+--------+-------------+----------------------+----------------+-----------------------+
|             3|1951/10/21 00:00:...|   F|          CA|   SANTA ROSA| RS|  GLORIA|98900000|              1|1998-10-28 00:00:00|1990/01/01 0

## Regra 1) Cadastrados a partir de 2017

In [8]:
cadastral_df = cadastral_df.where(cadastral_df["DATA_CADASTRO"] >= "2017-01-01 00:00:00")

## Regra 2) Cadastrados até dezembro de 2017

In [9]:
cadastral_df = cadastral_df.where(cadastral_df["DATA_CADASTRO"] < "2018-01-01 00:00:00")

In [11]:
cadastral_df.count()

178403

## Regra 3) Clientes que efetuaram compras até os 6 meses após o cadastro

# Contrato

In [ ]:
contrato = sc.textFile('CONTRATO.txt')

In [ ]:
header2 = contrato.first()
header2

In [ ]:
contrato = contrato.filter(lambda line: line != header2)

In [ ]:
contrato_df = contrato.map(lambda k: k.split("\t")).toDF(header2.split("\t"))
contrato_df.show(5)

# Cartão

In [ ]:
cartao = sc.textFile('CARTAO.txt')

In [ ]:
header3 = cartao.first()
header3

In [ ]:
cartao = cartao.filter(lambda line: line != header3)

In [ ]:
cartao_df = cartao.map(lambda k: k.split("\t")).toDF(header3.split("\t"))
cartao_df.show(5)

In [ ]:
cartao_df = cartao_df.withColumn("DATA_VENCIMENTO", to_timestamp("DATA_VENCIMENTO", "yyyy/MM/dd HH:mm:ss"))
cartao_df = cartao_df.withColumn("DATA_PAGAMENTO", to_timestamp("DATA_PAGAMENTO", "yyyy/MM/dd HH:mm:ss"))

In [ ]:
cartao_df.show(5)

# PAGAMENTO ANTES DO VENCIMENTO

In [ ]:
good_cartao = cartao_df.select(cartao_df['CODIGO_CLIENTE'],cartao_df['DATA_PAGAMENTO'],cartao_df['DATA_VENCIMENTO']).where(cartao_df['DATA_PAGAMENTO'] < cartao_df['DATA_VENCIMENTO'])

In [ ]:
good_cartao.show(5)

In [ ]:
possible_good = good_cartao.select('CODIGO_CLIENTE').dropDuplicates()

In [ ]:
possible_good.count()

In [ ]:
"""test = possible_good.select('CODIGO_CLIENTE').where(possible_good['CODIGO_CLIENTE'].isin(bad_client_cartao['CODIGO_CLIENTE']) == True)
test.count()"""

# PAGAMENTO DEPOIS DO VENCIMENTO

In [ ]:
bad_cartao = cartao_df.select(cartao_df['CODIGO_CLIENTE'],cartao_df['DATA_PAGAMENTO'],cartao_df['DATA_VENCIMENTO']).where(cartao_df['DATA_PAGAMENTO'] > cartao_df['DATA_VENCIMENTO'])

In [ ]:
bad_cartao.show(5)

In [ ]:
bad_client_cartao = bad_cartao.select('CODIGO_CLIENTE').dropDuplicates()

In [ ]:
bad_client_cartao.cache()

In [ ]:
bad_client_cartao.show(10)

In [ ]:
bad_client_cartao = bad_client_cartao.withColumn("CLASS", lit(0))

In [ ]:
bad_client_cartao.show()